In [2]:
from sim import make_sim
exp_dict, truths = make_sim('/hildafs/home/pladuca/main/skyCatalog.yaml',500, 'ir', 50, 4.800542855802589, 2.286575, 31.4,)

2025-11-24 19:07:16,372 - INFO - Catalog data will be read from /hildafs/home/pladuca/main/roman_rubin_cats_v1.1.2_faint/roman_rubin_cats_v1.1.2_faint/
2025-11-24 19:07:19,218 - INFO - Working on healpix pixel 10307
2025-11-24 19:07:22,523 - WARNING - In SkyCatalog.get_object_type_by_hp, healpix 10431  intersects region but has no data file for diffsky_galaxy
2025-11-24 19:07:22,524 - INFO - Working on healpix pixel 10307
2025-11-24 19:07:22,605 - WARNING - In SkyCatalog.get_object_type_by_hp, healpix 10431  intersects region but has no data file for star


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1259 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/1259 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [30]:
import lsst.afw.image as afw_image
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask
from lsst.meas.algorithms.detection import SourceDetectionTask
from lsst.meas.algorithms.subtractBackground import SubtractBackgroundTask
from lsst.meas.deblender import SourceDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.meas.algorithms import GaussianPsfFactory as psffac
import lsst.meas.extensions.gaap
import lsst.meas.modelfit
import lsst.geom as geom
from lsst.geom import Point2D
from lsst.afw.geom import makeSkyWcs
from lsst.afw.table import SourceTable
import galsim
import coord
import numpy as np
exp = exp_dict['i']
configDetection = SourceDetectionTask.ConfigClass() # all from Tae code
configDetection.thresholdValue = 3
# configDetection.thresholdType = "stdev" 
configDetection.doTempLocalBackground = False
configDetection.reEstimateBackground = False
# configDetection.nSigmaToGrow = 2.0
# configDetection.nSigmaForKernel = 10
# configDetection.minPixels = 4

configDeblend = SourceDeblendTask.ConfigClass()
configMeasurement = SingleFrameMeasurementTask.ConfigClass()
configMeasurement.plugins.names |= [
    "modelfit_DoubleShapeletPsfApprox",
    "modelfit_CModel",
    "ext_gaap_GaapFlux",
]
configMeasurement.slots.modelFlux = "modelfit_CModel"

schema = SourceTable.makeMinimalSchema()
raerr = schema.addField("coord_raErr", type="F")
decerr = schema.addField("coord_decErr", type="F")

detect = SourceDetectionTask(schema=schema, config=configDetection)
detect.canMultiprocess = True

deblender = SourceDeblendTask(schema=schema, config=configDeblend)
#background = SubtractBackgroundTask() ?
measure = SingleFrameMeasurementTask(schema=schema, config=configMeasurement)
measure.canMultiprocess = True

table = SourceTable.make(schema)
detect_result = detect.run(table=table, exposure=exp)
detected_catalog = detect_result.sources
deblender.run(exp, detected_catalog)
measure.run(measCat=detected_catalog, exposure=exp)
detected_catalog = detected_catalog.copy(True)

In [32]:
from lsst.afw.image import MultibandExposure
from lsst.pipe.tasks.multiBand import DetectCoaddSourcesTask, MeasureMergedCoaddSourcesTask
from lsst.pipe.tasks.mergeDetections import MergeDetectionsTask
from lsst.pipe.tasks.mergeMeasurements import MergeMeasurementsTask
from lsst.pipe.tasks.postprocess import WriteObjectTableTask
from lsst.drp.tasks.forcedPhotCoadd import ForcedPhotCoaddTask
from lsst.pipe.tasks.deblendCoaddSourcesPipeline import DeblendCoaddSourcesMultiTask
# from lsst.meas.extensions.scarlet import ScarletDeblendTask
from lsst.meas.base import SingleFrameMeasurementTask
from lsst.afw.table import IdFactory
from lsst.pipe.base import Struct
from lsst.afw.table import SourceTable
from lsst.afw.detection import PeakTable

ref_exp = exp_dict['i']
bbox = ref_exp.getBBox()
wcs = ref_exp.getWcs()

patchInfo = Struct(outerBBox=bbox, getOuterBBox=lambda: bbox)
skyInfo = Struct(wcs=wcs, patchInfo=patchInfo)

configDetection = DetectCoaddSourcesTask.ConfigClass()
configMergeDetect = MergeDetectionsTask.ConfigClass()
configMergeDetect.priorityList = ['i','r','g','z','y','u']
configDeblend = DeblendCoaddSourcesMultiTask.ConfigClass()

configMeasurement = MeasureMergedCoaddSourcesTask.ConfigClass()
configMeasurement.doPropagateFlags = False
configMeasurement.measurement.plugins.names |= [
    "modelfit_DoubleShapeletPsfApprox",
    "modelfit_CModel",
    #"ext_gaap_GaapFlux",
]
configMeasurement.measurement.slots.modelFlux = "modelfit_CModel"
configMeasurement.measurement.slots.psfShape = "base_SdssShape_psf"
# configMeasurement.mea
# configMeasurement.measurement.plugins['ext_gaap_GaapFlux'].doMeasure = True
# configMeasurement = SingleFrameMeasurementTask.ConfigClass()
# configMeasurement.plugins.names |= [
#     "modelfit_DoubleShapeletPsfApprox",
#     "modelfit_CModel",
#     "ext_gaap_GaapFlux",
# ]
# configMeasurement.slots.modelFlux = "modelfit_CModel"

configMergeMeasure = MergeMeasurementsTask.ConfigClass()
configMergeMeasure.priorityList = ['i','r','g','z','y','u']
configForcedPhot = ForcedPhotCoaddTask.ConfigClass()

configWrite = WriteObjectTableTask.ConfigClass()

schema = SourceTable.makeMinimalSchema()
# raerr = schema.addField("coord_raErr", type="F")
# decerr = schema.addField("coord_decErr", type="F")
# peakSchema = PeakTable.makeMinimalSchema()

detect = DetectCoaddSourcesTask(schema=schema, config=configDetection)
schema = detect.detectionSchema.schema

mergeDetect = MergeDetectionsTask(schema=schema, config=configMergeDetect)
schema = mergeDetect.outputSchema.schema
peakSchema = mergeDetect.outputPeakSchema.schema

initInputs = {'inputSchema':Struct(schema=schema), 'peakSchema':Struct(schema=peakSchema)}
deblend = DeblendCoaddSourcesMultiTask(initInputs, config=configDeblend)
schema = deblend.outputSchema.schema

measure = MeasureMergedCoaddSourcesTask(schema=schema, config=configMeasurement)
measure.setPrimaryFlags.isSingleFrame = True
schema = measure.outputSchema.schema

mergeMeasure = MergeMeasurementsTask(schema=schema, config=configMergeMeasure)
schema = mergeMeasure.outputSchema

forcedPhot = ForcedPhotCoaddTask(initInputs={'inputSchema':schema}, config=configForcedPhot)
schema = forcedPhot.outputSchema.schema

write = WriteObjectTableTask(config=configWrite)

idFactory = IdFactory.makeSimple()
ids = {'u':1, 'g':2,'r':3,'i':4,'z':5,'y':6}
sources = {}
bands = []
exposures = []
for band, exp in exp_dict.items():
    sources[band] = detect.run(exp, idFactory, ids[band]).outputSources
    bands.append(band)
    exposures.append(exp)
mergedDet = mergeDetect.run(sources, skyInfo=skyInfo, idFactory=idFactory,skySeed=67).outputCatalog
# mExposure = MultibandExposure.fromExposures(bands, exposures)
deblended = deblend.run(exposures,bands, mergedDet, idFactory).deblendedCatalog
print(deblended.getSchema())

Can't find flag ext_photometryKron_KronFlux_flag in schema: "Field with name 'ext_photometryKron_KronFlux_flag' not found"


Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['Flag'](name="merge_footprint_i", doc="Detection footprint overlapped with a detection from filter i"), Key['Flag'](offset=32, bit=0)),
    (Field['Flag'](name="merge_footprint_r", doc="Detection footprint overlapped with a detection from filter r"), Key['Flag'](offset=32, bit=1)),
    (Field['Flag'](name="merge_footprint_g", doc="Detection footprint overlapped with a detection from filter g"), Key['Flag'](offset=32, bit=2)),
    (Field['Flag'](name="merge_footprint_z", doc="Detection footprint overlapped with a detection from filter z"), Key['Flag'](offset=32, bit=3)),
    (Field['Flag'](name="m

In [34]:
deblended

<class 'lsst.afw.table.SourceCatalog'>
Non-contiguous afw.Catalog of 328 rows.
columns: id coord_ra coord_dec parent merge_footprint_i merge_footprint_r merge_footprint_g merge_footprint_z merge_footprint_y merge_footprint_u merge_footprint_sky merge_peak_i merge_peak_r merge_peak_g merge_peak_z merge_peak_y merge_peak_u merge_peak_sky deblend_runtime deblend_iterations deblend_nChild deblend_nPeaks deblend_skipped deblend_isolatedParent deblend_isPseudo deblend_tooManyPeaks deblend_parentTooBig deblend_masked deblend_sedConvergenceFailed deblend_morphConvergenceFailed deblend_blendConvergenceFailedFlag deblend_failed deblend_error deblend_incompleteData deblend_peak_center_x deblend_peak_center_y deblend_peakId deblend_peak_instFlux deblend_modelType deblend_parentNPeaks deblend_parentNChild deblend_scarletFlux deblend_logL deblend_edgePixels deblend_spectrumInitFlag deblend_nComponents deblend_deblendedAsPsf deblend_dataCoverage deblend_zeroFlux deblend_maxOverlap deblend_fluxOverlap

In [27]:
measurements = {}
for band, exp in exp_dict.items():
    measurements[band] = measure.run(exp, deblended, Struct(skyMap=None, tractInfo=None, patchInfo=None),ids[band])
mergeMes = mergeMeasure.run(measurements)
forcedResults = {}
for band, exp in exp_dict.items():
    forcedResults[band] = forcedPhot.run(measCat=mergeMes.sources,exposure=exp,refCat=mergeMes.sources,refWcs=exp.getWcs(),exposureId=ids[band])

out = write.run(forcedResults, 0, 0)

print(out)
print(out.columns)

AssertionError: 